In [197]:
import pandas as pd
import math
import numpy as np
from collections import Counter
import datetime

In [99]:
test_set = pd.read_excel('C:/DATASET_TEST/test_data_2022.xlsx')

In [100]:
test_set = test_set.set_index('row_id')

In [101]:
test_set.isna().mean().sort_values(ascending = False) #-оценка количества пустых значений (~24% пустых диагнозов)

icd_code                 0.239746
insured_id               0.000000
date_of_birth            0.000000
gender                   0.000000
employer_id              0.000000
medical_provider_code    0.000000
date_of_service          0.000000
service_name             0.000000
service_quantity         0.000000
service_amount           0.000000
dtype: float64

In [102]:
#-вычисление возраста страховщика на момент обращения
test_set['AGE'] = ((test_set['date_of_service']-test_set['date_of_birth']).dt.days)/365.25 

#test_set.head(25)

In [103]:
test_set["g_a_category"] = 'na' #-столбец с категориями клиентов

In [104]:
#-функция присваивает категорию клиенту, в зависимости от возраста и пола
def cat_age_gender(row):
    if row['gender'] == 'М' and row['AGE'] >= 0 and row['AGE'] < 18:
        return 'Дети-М'
    if row['gender'] == 'Ж' and row['AGE'] >= 0 and row['AGE'] < 18:
        return 'Дети-Ж'
    if row['gender'] == 'М' and row['AGE'] >= 18 and row['AGE'] < 60:
        return 'Взрослые-М'
    if row['gender'] == 'Ж' and row['AGE'] >= 18 and row['AGE'] < 60:
        return 'Взрослые-Ж'
    if row['gender'] == 'М' and row['AGE'] >= 60:
        return 'Пожилые-М'
    if row['gender'] == 'Ж' and row['AGE'] >= 60:
        return 'Пожилые-Ж'
    return row['g_a_category']

In [105]:
test_set["g_a_category"] = test_set.apply(cat_age_gender, axis=1) #применение функции ко всем строкам датафрейма

In [106]:
len(test_set)

23479

In [107]:
#сортировка данных по id клиента, коду мед.учреждения и дате обращения (для удобства работы)
test_set = test_set.sort_values(by=['insured_id','medical_provider_code','date_of_service'])

In [108]:
# добавляю столбец для уникальных номеров для каждого "CASE"
test_set['cases']=1

In [109]:
# для удобства, индексирую уже отсортированный df
test_set = test_set.reset_index()

In [110]:
# копирую номера index в отдельный столбец 'ind'
test_set['ind'] = test_set.index
test_set.head(10)

,row_id,insured_id,date_of_birth,gender,employer_id,medical_provider_code,date_of_service,service_name,service_quantity,icd_code,service_amount,AGE,g_a_category,cases,ind
0,32083270103,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-19,"Гепатит B, поверхностный антиген (HBs)Ag",1,K60.0,462.0,52.966461,Взрослые-М,1,0
1,32082978603,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-28,Инфильтрационная анестезия (при выполнении про...,4,K64.1,9408.0,52.991102,Взрослые-М,1,1
2,32082978903,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-29,"Прием (осмотр, консультация) врача-колопроктол...",1,K64.1,1078.0,52.993840,Взрослые-М,1,2
3,32082977803,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-30,Аппликационная анестезия (при выполнении прокт...,1,K64.1,553.0,52.996578,Взрослые-М,1,3
4,32178108603,746305716,1954-09-25,Ж,EMP-172-2206,MP-44-2977,2022-02-15,Рентгеноденситометрия проксимального отдела бе...,1,NaN,2244.0,67.392197,Пожилые-Ж,1,4
5,32209683903,746305716,1954-09-25,Ж,EMP-172-2206,MP-56-3842,2022-02-21,Массаж шеи при венозной дисфункции головного м...,1,M42,1017.0,67.408624,Пожилые-Ж,1,5
6,32183709303,746314816,1959-03-19,Ж,EMP-172-2206,MP-334-1729,2022-02-07,Консультация Кандидата медицинских наук,1,NaN,1700.0,62.891170,Пожилые-Ж,1,6
7,32183709203,746314816,1959-03-19,Ж,EMP-172-2206,MP-334-1729,2022-02-07,Исследование биомеханических свойств роговицы ...,2,NaN,1200.0,62.891170,Пожилые-Ж,1,7
8,32206982903,746334816,1971-12-13,М,EMP-172-2206,MP-12-1221,2022-01-25,Холестерин- ЛПВП (альфа-холестерин),1,I10,255.0,50.119097,Взрослые-М,1,8
9,32206982203,746334816,1971-12-13,М,EMP-172-2206,MP-12-1221,2022-01-25,Билирубин общий,1,I10,255.0,50.119097,Взрослые-М,1,9


In [111]:
len(test_set)

23479

In [112]:
#исходя из определения case, в столбец case, для каждой строки заносится номер case, к которому она относится
for index in test_set['ind']:
    if index == 0:
        pass
    elif (test_set.loc[index,'date_of_service'] - test_set.loc[index-1,'date_of_service']).days <=14 and test_set.loc[index,'medical_provider_code']==test_set.loc[index-1,'medical_provider_code'] and test_set.loc[index,'insured_id']==test_set.loc[index-1,'insured_id']:
        test_set.loc[index,'cases'] = test_set.loc[index-1,'cases']
    else:
        test_set.loc[index,'cases'] = test_set.loc[index-1,'cases']+1
       
    
    

In [113]:
test_set

,row_id,insured_id,date_of_birth,gender,employer_id,medical_provider_code,date_of_service,service_name,service_quantity,icd_code,service_amount,AGE,g_a_category,cases,ind
0,32083270103,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-19,"Гепатит B, поверхностный антиген (HBs)Ag",1,K60.0,462.0,52.966461,Взрослые-М,1,0
1,32082978603,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-28,Инфильтрационная анестезия (при выполнении про...,4,K64.1,9408.0,52.991102,Взрослые-М,1,1
2,32082978903,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-29,"Прием (осмотр, консультация) врача-колопроктол...",1,K64.1,1078.0,52.993840,Взрослые-М,1,2
3,32082977803,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-30,Аппликационная анестезия (при выполнении прокт...,1,K64.1,553.0,52.996578,Взрослые-М,1,3
4,32178108603,746305716,1954-09-25,Ж,EMP-172-2206,MP-44-2977,2022-02-15,Рентгеноденситометрия проксимального отдела бе...,1,NaN,2244.0,67.392197,Пожилые-Ж,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23474,32201712603,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-04,Клинический анализ мочи (физико-химические сво...,1,П,220.0,62.009582,Пожилые-Ж,10623,23474
23475,32201521203,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-10,Амплипульс без лекарственных веществ,9,M51.8,1530.0,62.026010,Пожилые-Ж,10623,23475
23476,32201713003,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-11,Консультация врача-терапевта высшей категории,1,П0,960.0,62.028747,Пожилые-Ж,10623,23476
23477,32217877003,1514194581303,2020-05-10,Ж,EMP-267-4626,MP-10-3785,2022-02-06,"Прием (осмотр, консультация) врача-офтальмолог...",1,H04.8,1908.0,1.744011,Дети-Ж,10624,23477


In [114]:
#столбец с уникальным номером визита, к которому относится строка
test_set['visit_id'] = 1

In [115]:
#для каждой строки проставляется номер визита, к которому она относится
for index in test_set['ind']:
    if index == 0:
        pass
    elif test_set.loc[index,'date_of_service'] == test_set.loc[index-1,'date_of_service'] and test_set.loc[index,'medical_provider_code'] == test_set.loc[index-1,'medical_provider_code'] and test_set.loc[index,'insured_id'] == test_set.loc[index-1,'insured_id']:
        test_set.loc[index,'visit_id'] = test_set.loc[index-1,'visit_id']
    else:
        test_set.loc[index,'visit_id'] = test_set.loc[index-1,'visit_id']+1

In [116]:
test_set

,row_id,insured_id,date_of_birth,gender,employer_id,medical_provider_code,date_of_service,service_name,service_quantity,icd_code,service_amount,AGE,g_a_category,cases,ind,visit_id
0,32083270103,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-19,"Гепатит B, поверхностный антиген (HBs)Ag",1,K60.0,462.0,52.966461,Взрослые-М,1,0,1
1,32082978603,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-28,Инфильтрационная анестезия (при выполнении про...,4,K64.1,9408.0,52.991102,Взрослые-М,1,1,2
2,32082978903,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-29,"Прием (осмотр, консультация) врача-колопроктол...",1,K64.1,1078.0,52.993840,Взрослые-М,1,2,3
3,32082977803,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-30,Аппликационная анестезия (при выполнении прокт...,1,K64.1,553.0,52.996578,Взрослые-М,1,3,4
4,32178108603,746305716,1954-09-25,Ж,EMP-172-2206,MP-44-2977,2022-02-15,Рентгеноденситометрия проксимального отдела бе...,1,NaN,2244.0,67.392197,Пожилые-Ж,2,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23474,32201712603,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-04,Клинический анализ мочи (физико-химические сво...,1,П,220.0,62.009582,Пожилые-Ж,10623,23474,15886
23475,32201521203,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-10,Амплипульс без лекарственных веществ,9,M51.8,1530.0,62.026010,Пожилые-Ж,10623,23475,15887
23476,32201713003,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-11,Консультация врача-терапевта высшей категории,1,П0,960.0,62.028747,Пожилые-Ж,10623,23476,15888
23477,32217877003,1514194581303,2020-05-10,Ж,EMP-267-4626,MP-10-3785,2022-02-06,"Прием (осмотр, консультация) врача-офтальмолог...",1,H04.8,1908.0,1.744011,Дети-Ж,10624,23477,15889


In [117]:
test_set['case_diag'] = 'n/a'
test_set.head(20)

,row_id,insured_id,date_of_birth,gender,employer_id,medical_provider_code,date_of_service,service_name,service_quantity,icd_code,service_amount,AGE,g_a_category,cases,ind,visit_id,case_diag
0,32083270103,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-19,"Гепатит B, поверхностный антиген (HBs)Ag",1,K60.0,462.0,52.966461,Взрослые-М,1,0,1,n/a
1,32082978603,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-28,Инфильтрационная анестезия (при выполнении про...,4,K64.1,9408.0,52.991102,Взрослые-М,1,1,2,n/a
2,32082978903,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-29,"Прием (осмотр, консультация) врача-колопроктол...",1,K64.1,1078.0,52.993840,Взрослые-М,1,2,3,n/a
3,32082977803,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-30,Аппликационная анестезия (при выполнении прокт...,1,K64.1,553.0,52.996578,Взрослые-М,1,3,4,n/a
4,32178108603,746305716,1954-09-25,Ж,EMP-172-2206,MP-44-2977,2022-02-15,Рентгеноденситометрия проксимального отдела бе...,1,NaN,2244.0,67.392197,Пожилые-Ж,2,4,5,n/a
5,32209683903,746305716,1954-09-25,Ж,EMP-172-2206,MP-56-3842,2022-02-21,Массаж шеи при венозной дисфункции головного м...,1,M42,1017.0,67.408624,Пожилые-Ж,3,5,6,n/a
6,32183709303,746314816,1959-03-19,Ж,EMP-172-2206,MP-334-1729,2022-02-07,Консультация Кандидата медицинских наук,1,NaN,1700.0,62.891170,Пожилые-Ж,4,6,7,n/a
7,32183709203,746314816,1959-03-19,Ж,EMP-172-2206,MP-334-1729,2022-02-07,Исследование биомеханических свойств роговицы ...,2,NaN,1200.0,62.891170,Пожилые-Ж,4,7,7,n/a
8,32206982903,746334816,1971-12-13,М,EMP-172-2206,MP-12-1221,2022-01-25,Холестерин- ЛПВП (альфа-холестерин),1,I10,255.0,50.119097,Взрослые-М,5,8,8,n/a
9,32206982203,746334816,1971-12-13,М,EMP-172-2206,MP-12-1221,2022-01-25,Билирубин общий,1,I10,255.0,50.119097,Взрослые-М,5,9,8,n/a


In [118]:
test_set2 = test_set.dropna(subset = ['icd_code']).copy() #DF для работы с диагнозами, без NaN значений
def case_diag(row):
    lst = list(test_set2[test_set2['cases']==row['cases']]['icd_code'])
    counts = Counter(lst)
    return (sorted(lst, key=lambda x: -counts[x]))[0]
       

In [119]:
test_set2['case_diag'] = test_set2.apply(case_diag, axis=1)

In [120]:
test_set2

,row_id,insured_id,date_of_birth,gender,employer_id,medical_provider_code,date_of_service,service_name,service_quantity,icd_code,service_amount,AGE,g_a_category,cases,ind,visit_id,case_diag
0,32083270103,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-19,"Гепатит B, поверхностный антиген (HBs)Ag",1,K60.0,462.0,52.966461,Взрослые-М,1,0,1,K64.1
1,32082978603,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-28,Инфильтрационная анестезия (при выполнении про...,4,K64.1,9408.0,52.991102,Взрослые-М,1,1,2,K64.1
2,32082978903,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-29,"Прием (осмотр, консультация) врача-колопроктол...",1,K64.1,1078.0,52.993840,Взрослые-М,1,2,3,K64.1
3,32082977803,746279116,1969-01-31,М,EMP-1684-4513,MP-10-3785,2022-01-30,Аппликационная анестезия (при выполнении прокт...,1,K64.1,553.0,52.996578,Взрослые-М,1,3,4,K64.1
5,32209683903,746305716,1954-09-25,Ж,EMP-172-2206,MP-56-3842,2022-02-21,Массаж шеи при венозной дисфункции головного м...,1,M42,1017.0,67.408624,Пожилые-Ж,3,5,6,M42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23474,32201712603,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-04,Клинический анализ мочи (физико-химические сво...,1,П,220.0,62.009582,Пожилые-Ж,10623,23474,15886,M42.
23475,32201521203,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-10,Амплипульс без лекарственных веществ,9,M51.8,1530.0,62.026010,Пожилые-Ж,10623,23475,15887,M42.
23476,32201713003,1514079369803,1960-02-01,Ж,EMP-1197-3588,MP-527-2871,2022-02-11,Консультация врача-терапевта высшей категории,1,П0,960.0,62.028747,Пожилые-Ж,10623,23476,15888,M42.
23477,32217877003,1514194581303,2020-05-10,Ж,EMP-267-4626,MP-10-3785,2022-02-06,"Прием (осмотр, консультация) врача-офтальмолог...",1,H04.8,1908.0,1.744011,Дети-Ж,10624,23477,15889,H04.8


In [ ]:
#столбец с группой заболеваний
test_set2['diag_group'] = 'n/a'
    

In [152]:
#функция, заполняющая столбец группы заболеваний по первому символу диагноза кейса
def diag_group(row):
    return (row['case_diag'])[0]

In [159]:
#(test_set2[test_set2['ind']==328]['case_diag']).values.astype(str)[0][0]

In [154]:
test_set2['diag_group'] = test_set2.apply(diag_group,axis=1)

In [157]:
#test_set2

In [158]:
#test_set

In [179]:
test_set['case_length'] = 0
test_set2['case_length'] = 0

In [198]:
def case_len(row):
    dd = list(test_set[test_set['cases']==row['cases']]['date_of_service'])
    ff =(dd[-1] - dd[0]) + datetime.timedelta(days=1)
    return ff
    

In [199]:
test_set['case_length'] = test_set.apply(case_len, axis=1)

In [246]:
mmm = test_set.groupby(["cases","case_length"])["service_amount"].sum()

In [249]:
len(mmm)

10624